In [34]:
from pathlib import Path
import tifffile
import napari
import os
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
from utils import list_images, read_image, simulate_cytoplasm

In [35]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
markers = [("ki_67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

In [36]:
# Explore each image to analyze (0 defines the first image in the directory)
image = images[0]

# Remember to input the same slicing factor you used when generating the nuclei predictions
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Generate maximum intensity projection and extract filename
img_mip, filename = read_image(image, slicing_factor)

# Extract filename with extension
file_and_ext = Path(image).stem

# Show image in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

Image displayed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)


<Image layer 'img_mip' at 0x22a276a2eb0>

In [37]:
# List of subfolder names
roi_names = [folder.name for folder in nuclei_preds_path.iterdir() if folder.is_dir()]
print(f"The following regions of interest will be analyzed: {roi_names}")

The following regions of interest will be analyzed: ['CA', 'DG', 'full_image']


In [38]:
# Initialize an empty list to hold the extracted dataframes on a per ROI basis
per_roi_props = []

for roi_name in roi_names:

    # Initialize an empty list to hold the extracted dataframes on a per channel basis
    props_list = []

    # Read the nuclei predictions per ROI
    nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{file_and_ext}.tiff")

    # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
    try:
        user_roi = tifffile.imread(roi_path / roi_name / f"{file_and_ext}.tiff")
    except FileNotFoundError:
        # Extract the xy dimensions of the input image (from nuclei_labels)
        img_shape = nuclei_labels.shape
        img_xy_dims = img_shape[-2:]

        # Create a label covering the entire image
        user_roi = np.ones(img_xy_dims).astype(np.uint8)

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(nuclei_labels, name=f"{roi_name}_nuclei")

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(user_roi, name=f"{roi_name}_ROI", opacity=0.4)

    # Create a dictionary containing all image descriptors
    descriptor_dict = {
                "filename": filename,
                "ROI": roi_name,
                }

    # Loop through each channel and extract the average intensity within either nuclei or cytoplasmic regions
    for tuple in markers:

        channel_name = tuple[0]
        ch_nr = tuple[1]
        location = tuple[2]

        if location == "cytoplasm":

            # Simulate a cytoplasm by dilating the nuclei and substracting the nuclei mask afterwards
            cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

            # Add the predicted nuclei as labels into Napari
            viewer.add_labels(cytoplasm_labels, name=f"{roi_name}_cytoplasm")

            # Extract intensity information from each marker channel
            props = regionprops_table(label_image=cytoplasm_labels,
                                intensity_image=img_mip[ch_nr],
                                properties=["label", "intensity_mean"])
        
        elif location == "nucleus":

            # Extract intensity information from each marker channel
            props = regionprops_table(label_image=nuclei_labels,
                                intensity_image=img_mip[ch_nr],
                                properties=["label", "intensity_mean"])
        

        # Convert to dataframe
        props_df = pd.DataFrame(props)

        # Rename intensity_mean column to indicate the specific image
        props_df.rename(columns={"intensity_mean": f"{location}_{channel_name}_avg_int"}, inplace=True)

        # Append each props_df to props_list
        props_list.append(props_df)

        # Initialize the df with the first df in the list
        props_df = props_list[0]
        # Start looping from the second df in the list
        for df in props_list[1:]:
            props_df = props_df.merge(df, on="label")

    # Add each key-value pair from descriptor_dict to props_df at the specified position
    insertion_position = 0    
    for key, value in descriptor_dict.items():
        props_df.insert(insertion_position, key, value)
        insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

    # Append each props_df to props_list
    per_roi_props.append(props_df)



In [39]:
final_df = pd.concat(per_roi_props, ignore_index=True)

final_df

,filename,ROI,label,nucleus_ki_67_avg_int,nucleus_neun_avg_int,cytoplasm_calbindin_avg_int
0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,1,93.302419,100.520161,12.485149
1,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,2,109.193662,160.524648,14.086705
2,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,3,86.524465,91.558104,15.710280
3,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,4,60.469697,15.094697,8.805755
4,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,5,86.252907,97.027132,15.851145
...,...,...,...,...,...,...
8127,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,full_image,5050,83.953488,7.519380,20.267606
8128,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,full_image,5051,132.551724,21.896552,88.683333
8129,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,full_image,5052,105.606061,9.878788,19.132075
8130,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,full_image,5053,180.428571,24.795918,27.459184


In [40]:
# Create a 'results' folder in the root directory
results_folder = 'results'

try:
    os.makedirs(results_folder)
    print(f"'{results_folder}' folder created successfully.")
except FileExistsError:
    print(f"'{results_folder}' folder already exists.")

# Save the df containing per_label results into a CSV file
final_df.to_csv(f'./results/{filename}_per_label.csv')

'results' folder already exists.


In [ ]:
#TODO: Show positive cells based on average intensity threshold, make it usable also based on cluster numbers i.e. for Neun
# Filter dataframe based on threshold, recover labels, remove from labels and show surviving ones in Napari
# Loop over regions or ask user for a region in particular